# Show ranking tables of simulations against experimental data

This notebook shows different kinds of rankings of simulations against experimental data. This reading ranking tables from '../../Data/Ranking/' folder. Before using this, run 'python makeRanking.py' in 'Scripts/BuildDatabank' folder.

# Initialize NMRlipids databank

In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
#import mdtraj
import urllib.request
import yaml
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import math
import numpy as np

sys.path.insert(1, '../BuildDatabank/')
from databankLibrary import *

path = '../../Data/Simulations/'
db_data = databank(path)
systems = db_data.get_systems()

# Show ranking of simulations based on total quality, tail quality and headgroup quality

In [2]:
Fragments = ['total','tails','headgroup','FormFactor']

for SortBasedOn in Fragments:
    #print(SortBasedOn)

    print('Sorted based on ', SortBasedOn, ' quality')
    
    FFrankingPath = '../../Data/Ranking/SYSTEM_' + SortBasedOn + '_Ranking.json'
    
    with open(FFrankingPath) as json_file:
        FFranking = json.load(json_file)
    json_file.close()

    ShowTable(FFranking,'TotalQuality')

Sorted based on  total  quality


,headgroup,tails,total,FFQuality,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.76,0.86,0.83,0.15,OPLS3e,POPC:SOL,(200:8859),300.00,1
1,0.65,0.84,0.78,0.08,Slipids,POPC:SOL,(122:4880),303.00,376
2,0.61,0.83,0.76,0.45,Slipids,POPC:SOL,(512:23943),298.00,617
3,0.58,0.81,0.73,0.45,Slipids,POPC:SOL,(1024:51200),298.15,696
4,0.70,0.73,0.72,0.65,MacRog,POPC:SOL,(1024:51200),298.15,658
5,0.63,0.73,0.69,0.55,MacRog,POPC:SOL,(128:5120),300.00,457
6,0.54,0.76,0.69,0.55,Slipids,POPC:SOL,(256:12800),298.15,708
7,0.51,0.77,0.68,0.22,Slipids2020,POPC:SOL,(128:5120),303.00,591
8,0.67,0.69,0.68,0.55,MacRog,POPC:SOL,(256:12800),298.15,675
9,0.62,0.71,0.68,0.55,MacRog,POPC:SOL,(64:3200),298.15,674


Sorted based on  tails  quality


,headgroup,tails,total,FFQuality,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.01,0.86,0.58,0.95,Berger,POPC:SOL,(256:10342),300.00,115
1,0.76,0.86,0.83,0.15,OPLS3e,POPC:SOL,(200:8859),300.00,1
2,0.65,0.84,0.78,0.08,Slipids,POPC:SOL,(122:4880),303.00,376
3,0.61,0.83,0.76,0.45,Slipids,POPC:SOL,(512:23943),298.00,617
4,0.01,0.83,0.55,0.88,Berger,POPC:SOL,(256:10342),303.00,151
5,0.58,0.81,0.73,0.45,Slipids,POPC:SOL,(1024:51200),298.15,696
6,0.02,0.81,0.55,1.05,Berger,POPC:SOL,(128:7290),298.00,497
7,0.01,0.78,0.52,0.10,Slipids,POPE:SOL,(500:25000),310.00,414
8,0.51,0.77,0.68,0.22,Slipids2020,POPC:SOL,(128:5120),303.00,591
9,0.54,0.76,0.69,0.55,Slipids,POPC:SOL,(256:12800),298.15,708


Sorted based on  headgroup  quality


,headgroup,tails,total,FFQuality,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.76,0.86,0.83,0.15,OPLS3e,POPC:SOL,(200:8859),300.00,1
1,0.70,0.61,0.64,1.02,CHARMM36,POPC:SOL,(128:5120),303.00,462
2,0.70,0.73,0.72,0.65,MacRog,POPC:SOL,(1024:51200),298.15,658
3,0.69,0.49,0.56,1.12,CHARMM36,POPC:SOL,(128:5120),303.00,123
4,0.69,0.56,0.60,1.02,CHARMM36,POPC:SOL,(128:4040),303.00,281
5,0.69,0.42,0.51,1.12,CHARMM36 noNBFIX,POPC:SOL,(200:9000),303.00,90
6,0.68,0.58,0.61,1.02,CHARMM36,POPC:SOL,(128:5120),303.00,98
7,0.68,0.58,0.62,1.02,CHARMM36,POPC:SOL,(128:5120),303.00,361
8,0.68,0.47,0.54,1.12,CHARMM36,POPC:SOL,(128:5120),303.00,301
9,0.68,0.59,0.62,1.02,CHARMM36,POPC:SOL,(128:5120),303.00,554


Sorted based on  FormFactor  quality


,headgroup,tails,total,FFQuality,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.56,0.57,0.57,0.02,Lipid14,POPC:SOL,(72:2234),303.00,154
1,NaN,NaN,NaN,0.07,ECC-lipids,POPC:SOL,(128:6400),313.00,360
2,NaN,NaN,NaN,0.07,Lipid17,POPC:SOL:CHOL,(64:4000:16),298.15,662
3,0.65,0.84,0.78,0.08,Slipids,POPC:SOL,(122:4880),303.00,376
4,NaN,NaN,NaN,0.10,Poger GROMOS 53A6_L,DPPC:SOL,(128:5841),323.00,365
5,NaN,NaN,NaN,0.10,Poger GROMOS 53A6_L,DPPC:SOL,(128:5841),323.00,593
6,0.01,0.78,0.52,0.10,Slipids,POPE:SOL,(500:25000),310.00,414
7,NaN,NaN,NaN,0.13,ECC-lipids,POPC:SOL,(128:6400),313.00,103
8,0.47,0.66,0.60,0.15,Lipid17,POPC:SOL,(64:3200),298.15,715
9,0.76,0.86,0.83,0.15,OPLS3e,POPC:SOL,(200:8859),300.00,1


# Show ranking separately for each lipid based on total, sn-1, sn-2, or headgroup quality

In [3]:
Fragments = ['total','sn-1','sn-2','headgroup']

for SortBasedOn in Fragments:
    for lipid in lipids_dict:
        print('Quality of',SortBasedOn,' of ',lipid)
        FFrankingPath = '../../Data/Ranking/' + lipid + '_' + SortBasedOn + '_Ranking.json'
    
        try:
            with open(FFrankingPath) as json_file:
                FFranking = json.load(json_file)
            json_file.close()
        except:
            print('File not found')
            continue
            
        ShowTable(FFranking,lipid)

Quality of total  of  POPC


,sn-1,sn-2,headgroup,total,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.87,0.85,0.76,0.83,OPLS3e,POPC:SOL,(200:8859),300.00,1
1,0.86,0.82,0.65,0.78,Slipids,POPC:SOL,(122:4880),303.00,376
2,0.84,0.82,0.61,0.76,Slipids,POPC:SOL,(512:23943),298.00,617
3,0.70,0.92,0.58,0.73,Slipids,POPC:SOL,(1024:51200),298.15,696
4,0.70,0.76,0.70,0.72,MacRog,POPC:SOL,(1024:51200),298.15,658
5,0.70,0.76,0.63,0.69,MacRog,POPC:SOL,(128:5120),300.00,457
6,0.63,0.89,0.54,0.69,Slipids,POPC:SOL,(256:12800),298.15,708
7,0.68,0.86,0.51,0.68,Slipids2020,POPC:SOL,(128:5120),303.00,591
8,0.65,0.73,0.67,0.68,MacRog,POPC:SOL,(256:12800),298.15,675
9,0.70,0.73,0.62,0.68,MacRog,POPC:SOL,(64:3200),298.15,674


Quality of total  of  POPG


,sn-1,sn-2,headgroup,total,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.1,0.48,0.12,0.23,Slipids,POPG:SOL:SOD,(288:10664:288),298.0,207


Quality of total  of  POPS


...

Quality of headgroup  of  POPE


,sn-1,sn-2,headgroup,total,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.52,0.27,0.54,0.44,CHARMM36,SOL:POPE,(5760:144),310.0,430
1,0.14,0.07,0.53,0.25,CHARMM36,SOL:POPE,(25000:500),310.0,67
2,0.56,0.26,0.32,0.38,CHARMM36-UA,POPE:SOL,(336:15254),310.0,352
3,0.80,0.58,0.30,0.56,Charmm-Drude polarizable2023,POPE:SOL,(72:2304),308.0,747
4,0.74,0.56,0.30,0.53,Charmm-Drude polarizable2023,POPE:SOL,(72:2304),308.0,750
5,0.83,0.48,0.29,0.53,GROMOS-CKP,POPE:SOL,(500:25000),310.0,400
6,0.60,0.54,0.28,0.48,Charmm-Drude polarizable2023,POPE:SOL,(72:2304),308.0,751
7,0.57,0.28,0.27,0.37,CHARMM36-UA,POPE:SOL,(336:15254),310.0,233
8,0.63,0.49,0.27,0.46,GROMOS-CKP,POPE:SOL,(128:3552),313.0,482
9,0.77,0.49,0.22,0.49,GROMOS-CKP,POPE:SOL,(128:3552),313.0,328


Quality of headgroup  of  PYPC
File not found
Quality of headgroup  of  PAzePCprot
File not found
Quality of headgroup  of  PAzePCdeprot
File not found
Quality of headgroup  of  DMPC
File not found
Quality of headgroup  of  DPPC
File not found
Quality of headgroup  of  DPPE
File not found
Quality of headgroup  of  DPPG
File not found
Quality of headgroup  of  DEPC
File not found
Quality of headgroup  of  DRPC
File not found
Quality of headgroup  of  DYPC
File not found
Quality of headgroup  of  DLPC
File not found
Quality of headgroup  of  DLIPC
File not found
Quality of headgroup  of  DOG
File not found
Quality of headgroup  of  DOPC


,sn-1,sn-2,headgroup,total,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.6,0.54,0.6,0.58,AMOEBA,DOPC:SOL,(72:2880),303,742


Quality of headgroup  of  DOPE
File not found
Quality of headgroup  of  DDOPC
File not found
Quality of headgroup  of  DOPS
File not found
Quality of headgroup  of  DSPC
File not found
Quality of headgroup  of  DAPC
File not found
Quality of headgroup  of  DMTAP
File not found
Quality of headgroup  of  SDG
File not found
Quality of headgroup  of  SDPE
File not found
Quality of headgroup  of  SOPC
File not found
Quality of headgroup  of  POPI
File not found
Quality of headgroup  of  SAPI
File not found
Quality of headgroup  of  SLPI
File not found
Quality of headgroup  of  CER
File not found
Quality of headgroup  of  CHOL
File not found
Quality of headgroup  of  DCHOL
File not found
Quality of headgroup  of  DHMDMAB
File not found
Quality of headgroup  of  SLiPC
File not found
Quality of headgroup  of  SM16
File not found


# Show form factor ranking only for systems with a specific lipid (cholesterol as an example)

In [4]:
### This is showing the ranking only for systems containing cholesterol

FFrankingPath = '../../Data/Ranking/SYSTEM_FormFactor_Ranking.json'
lipid = 'CHOL'

with open(FFrankingPath) as json_file:
    FFranking = json.load(json_file)
json_file.close()

NewRank = []
for i in FFranking:
    if lipid in i['system']['COMPOSITION']:
        NewRank.append(i)
   
ShowTable(NewRank,'TotalQuality')

,FFQuality,Forcefield,Molecules,Number of molecules,Temperature,ID,headgroup,tails,total
0,0.07,Lipid17,POPC:SOL:CHOL,(64:4000:16),298.15,662,NaN,NaN,NaN
1,0.17,Lipid17,POPC:SOL:CHOL,(1024:64000:256),298.15,667,NaN,NaN,NaN
2,0.17,Lipid17,POPC:SOL:CHOL,(256:16000:64),298.15,666,NaN,NaN,NaN
3,0.23,MacRog,POPC:SOL:CHOL,(1024:64000:256),298.15,677,NaN,NaN,NaN
4,0.23,MacRog,POPC:SOL:CHOL,(256:16000:64),298.15,706,NaN,NaN,NaN
5,0.26,Lipid17,POPC:SOL:CHOL,(64:3600:8),298.15,680,NaN,NaN,NaN
6,0.26,MacRog,POPC:SOL:CHOL,(64:3600:8),298.15,665,NaN,NaN,NaN
7,0.27,Berger and Modified Höltje model for cholesterol,POPC:CHOL:SOL,(110:18:8481),298.00,589,0.02,0.40,0.27
8,0.33,Berger and Modified Höltje model for cholesterol,POPC:CHOL:SOL,(84:44:6794),298.00,206,0.04,0.07,0.06
9,0.36,MacRog,POPC:SOL:CHOL,(1024:57600:128),298.15,660,NaN,NaN,NaN


# Show sn-1 ranking only for systems with POPC and CHOLESTEROL

In [5]:
### This is showing the ranking only for systems containing POPC and cholesterol

FFrankingPath = '../../Data/Ranking/POPC_sn-1_Ranking.json'
lipid = 'CHOL'


with open(FFrankingPath) as json_file:
    FFranking = json.load(json_file)
json_file.close()

NewRank = []

for i in FFranking:
    #print(i)
    #for tst in i:
    #    print(tst)
    if lipid in i['system']['COMPOSITION']:
        NewRank.append(i)
   
ShowTable(NewRank,'POPC')


,sn-1,sn-2,headgroup,total,Forcefield,Molecules,Number of molecules,Temperature,ID
0,0.86,0.55,0.58,0.66,Lipid14,POPC:CHOL:SOL,(120:8:3968),303.0,566
1,0.82,0.50,0.03,0.45,Berger and Modified Höltje model for cholesterol,POPC:CHOL:SOL,(120:8:7290),298.0,305
2,0.76,0.51,0.59,0.62,Lipid14,POPC:CHOL:SOL,(108:20:3968),303.0,369
3,0.73,0.52,0.74,0.66,Slipids,CHOL:POPC:SOL,(122:122:9760),303.0,250
4,0.65,0.57,0.56,0.59,Lipid14,POPC:CHOL:SOL,(64:64:3968),303.0,85
5,0.60,0.49,0.71,0.60,slipids,CHOL:POPC:SOL,(256:256:20334),298.0,82
6,0.47,0.45,0.61,0.51,CHARMM36,POPC:CHOL:SOL,(80:80:7200),303.0,451
7,0.44,0.39,0.64,0.49,CHARMM36,CHOL:POPC:SOL,(122:122:9760),303.0,13
8,0.42,0.57,0.62,0.53,CHARMM36,CHOL:POPC:SOL,(80:80:4496),303.0,9
9,0.37,0.71,0.03,0.37,Berger and Modified Höltje model for cholesterol,POPC:CHOL:SOL,(110:18:8481),298.0,589
